In [46]:
import pandas as pd
import math
import cv2
import numpy as np

In [30]:
df = pd.read_csv('../assets/csv/player_positions.csv')

In [31]:
df.head()

,frame,timestamp,track_id,x_pixel,y_pixel,x_normalized,y_normalized,width,height,confidence
0,141,4.7,1,322.371857,527.692749,0.251853,0.732907,31.998596,74.292633,0.812247
1,141,4.7,2,704.387451,379.023560,0.550303,0.526422,24.574646,55.304474,0.811213
2,141,4.7,3,512.187988,534.789490,0.400147,0.742763,36.113464,73.262085,0.752820
3,141,4.7,4,624.969299,440.231934,0.488257,0.611433,24.177490,58.095337,0.724814
4,141,4.7,5,361.496918,430.055847,0.282419,0.597300,29.135254,62.459686,0.673488


In [35]:
def distance (x1, y1, x2, y2):
    return math.sqrt((x2-x1)**2 + (y2-y1)**2)

In [41]:
radii = []
for time in df['timestamp'].unique():
    for index, row in df[df['timestamp'] == time].iterrows():
        centre = (row['x_pixel'], row['y_pixel'])
        dist = []
        for i, r in df[(df['timestamp'] == time) & (df['x_pixel'] != centre[0]) & (df['y_pixel'] != centre[1])].iterrows():
            temp_dist = distance(centre[0], centre[1], r['x_pixel'], r['y_pixel'])
            dist.append(temp_dist/2)
        if len(dist) == 0:
            dist.append(0)
        print(row['track_id']," has radius ", min(dist))
        radii.append(min(dist))
    print()

1.0  has radius  52.5921453378238
2.0  has radius  28.527341370270378
3.0  has radius  45.50563902221153
4.0  has radius  50.13408993571967
5.0  has radius  42.2180462036764
6.0  has radius  28.015756938061045
7.0  has radius  48.05437871975963
8.0  has radius  60.57896478867327
9.0  has radius  82.37945418513455
10.0  has radius  42.2180462036764
11.0  has radius  45.50563902221153
12.0  has radius  82.37945418513455
13.0  has radius  28.015756938061045
14.0  has radius  48.05437871975963

1.0  has radius  52.71938958873615
2.0  has radius  27.980375340919736
3.0  has radius  49.00821931548098
4.0  has radius  50.433818121022746
5.0  has radius  41.86993740162468
6.0  has radius  27.4535365506699
7.0  has radius  47.40910699684818
8.0  has radius  60.156918443305806
9.0  has radius  85.39386504379732
10.0  has radius  41.86993740162468
11.0  has radius  49.00821931548098
12.0  has radius  85.39386504379732
13.0  has radius  27.4535365506699
14.0  has radius  47.40910699684818

1.0  ha

In [42]:
df['radius'] = radii

In [45]:
df.head()

,frame,timestamp,track_id,x_pixel,y_pixel,x_normalized,y_normalized,width,height,confidence,radius
0,141,4.7,1,322.371857,527.692749,0.251853,0.732907,31.998596,74.292633,0.812247,52.592145
1,141,4.7,2,704.387451,379.023560,0.550303,0.526422,24.574646,55.304474,0.811213,28.527341
2,141,4.7,3,512.187988,534.789490,0.400147,0.742763,36.113464,73.262085,0.752820,45.505639
3,141,4.7,4,624.969299,440.231934,0.488257,0.611433,24.177490,58.095337,0.724814,50.134090
4,141,4.7,5,361.496918,430.055847,0.282419,0.597300,29.135254,62.459686,0.673488,42.218046


In [47]:
max_x = int(df['x_pixel'].max()) + 100
max_y = int(df['y_pixel'].max()) + 100

In [48]:
unique_times = sorted(df['timestamp'].unique())

In [51]:
for time in unique_times:
    # 1. Create a black background image
    # Shape is (Height, Width, Color Channels)
    frame = np.zeros((max_y, max_x, 3), dtype=np.uint8)
    
    # 2. Get data
    current_data = df[df['timestamp'] == time]
    
    # 3. Draw on the frame
    for index, row in current_data.iterrows():
        center = (int(row['x_pixel']), int(row['y_pixel']))
        radius = int(row['radius'])
        
        # Draw the center (Green dot)
        cv2.circle(frame, center, 2, (0, 255, 0), -1)
        
        # Draw the radius (Red circle)
        if radius > 0:
            cv2.circle(frame, center, radius, (0, 0, 255), 1)
            
        # Add ID text (optional)
        cv2.putText(frame, str(row['track_id']), (center[0]+5, center[1]), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.3, (255, 255, 255), 1)

    # 4. Show the frame
    cv2.imshow('Tracking Screen', frame)
    
    # 5. Wait for key press (1ms = approx 1000 fps, adjust wait time for speed)
    # Press 'q' to quit
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()